In [62]:
from datetime import date
# get todays date
today = date.today()
print("Today's date:", today)

# get the date 2 years ago
two_years_ago = today.replace(year=today.year - 2)
print("Two years ago:", two_years_ago)

Today's date: 2023-11-09
Two years ago: 2021-11-09


In [70]:
import requests
import pandas as pd

url = f"https://api.polygon.io/v2/aggs/ticker/C:USDCAD/range/1/day/{two_years_ago}/{today}?adjusted=true&sort=asc&limit=50000&apiKey=I3RTEm6vso7yOXBhGcYSidwUhRHaSgWy"

response = requests.get(url).json()
cad_data = response['results']
# create a dataframe from the data
cad_df = pd.DataFrame(cad_data)
# convert the timestamp to a date and set it as the index
cad_df['date'] = pd.to_datetime(cad_df['t'], unit='ms')
cad_df.set_index('date', inplace=True)
cad_df


,v,vw,o,c,h,l,t,n
date,,,,,,,,
2021-11-10,137646,1.2444,1.24353,1.24863,1.25031,1.23850,1636502400000,137646
2021-11-11,123292,1.2550,1.24865,1.25885,1.25962,1.24790,1636588800000,123292
2021-11-12,101171,1.2578,1.25886,1.25315,1.26045,1.25108,1636675200000,101171
2021-11-14,1591,1.2546,1.25449,1.25453,1.25505,1.25353,1636848000000,1591
2021-11-15,102657,1.2524,1.25455,1.25140,1.25558,1.24990,1636934400000,102657
...,...,...,...,...,...,...,...,...
2023-11-05,1348,1.3659,1.36491,1.36606,1.36711,1.36455,1699142400000,1348
2023-11-06,127813,1.3662,1.36600,1.36960,1.37050,1.36270,1699228800000,127813
2023-11-07,136882,1.3743,1.36975,1.37670,1.37824,1.36920,1699315200000,136882


In [67]:
# use ploty to plot the data using candlestick
import plotly.graph_objects as go

# create a candlestick chart
fig = go.Figure(data=[go.Candlestick(x=cad_df.index,
                open=cad_df['o'],
                high=cad_df['h'],
                low=cad_df['l'],
                close=cad_df['c'])])

# do not display the range slider
fig.update_layout(xaxis_rangeslider_visible=False)

# display the chart
fig.show()


In [68]:
# get the data from yahoo finance
def get_data(symbol):
    # todo: add a doc string
    # get data from yahoo finance to use
    symbol_data = yf.download(symbol, period='max', rounding=False, prepost=True)
    return symbol_data

# splice the data when povided a date best to do a forecast on 2 years of data
def splice_data(df, date, query=False, query_on=''):
    # todo: add a doc string
    if query:
        return df.query(f'{query_on} >= "{date}"')
    return df.loc[date:]    

def forcasting_preparation(df):
    # todo: add a doc string
    df = df.reset_index()
    return df[['Date', 'Close']]
        
# use prophet to forecast the data
def forecast_data(data):
    # todo: add a doc string
    data = data.rename(columns={'Date': 'ds', 'Close': 'y'})
    model = Prophet()
    model.fit(data)
    future = model.make_future_dataframe(periods=90, freq='D')
    forecast = model.predict(future)
    return forecast

def process_forecasted_data(forecast_df):
    df = forecast_df.copy()
    # keep only needed columns in the forecast dataframe
    df = df[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'trend']]  
    # smooth out the prediction lines
    df['predicted_price'] = df['yhat'].rolling(window=7).mean()
    df['upper_band'] = df['yhat_upper'].rolling(window=7).mean()
    df['lower_band'] = df['yhat_lower'].rolling(window=7).mean()
    return df

def plotly_visualize_forecast(symbol, data, forcast_processed, width=1400, height=800):
    #  get timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d @ %H:%M:%S")
    date_buttons = [{'count': 9, 'label': '6M', 'step': "month", 'stepmode': "todate"},
                    {'count': 6, 'label': '3M', 'step': "month", 'stepmode': "todate"},
                    {'count': 4, 'label': '1M', 'step': "month", 'stepmode': "todate"}]
    # create the plotly chart
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=data.index, open=data.Open, high=data.High, low=data.Low, close=data.Close, name='Candlestick', increasing_line_color='#F6FEFF', decreasing_line_color='#1CBDFB'))
    
    # update the layout of the chart with the buttons and timestamp along with some kwargs
    fig.update_layout(  
        {'xaxis':
            {'rangeselector': {'buttons': date_buttons, 
                                'bgcolor': '#444654', 
                                'activecolor': '#1E82CD',
                                'bordercolor': '#444654',
                                'font': {'color': 'white'}}
            }
        },
            width=width, height=height, xaxis_rangeslider_visible=False, 
            paper_bgcolor='#202123', plot_bgcolor='#202123', font=dict(color='white', size=12),
            font_size=14, font_family="Rockwell", title_font_family="Rockwell", title_font_size=24
    )
    
    #  update the layout of the chart with the title and axis labels
    fig.update_layout( 
        {'annotations': [{  "text": f"This graph was last generated on {timestamp}", 
                            "showarrow": False, "x": 0.55, "y": 1.05, "xref": "paper", "yref": "paper"}]},
    )

    fig.update_layout( 
        {'title': {'text':f'{symbol} Price Chart', 'x': 0.5, 'y': 0.95}},
        yaxis=dict(title='Price', gridcolor='#444654'), xaxis=dict(gridcolor='#444654')
    )
    # Update y-axes to include dollar sign
    fig.update_yaxes(tickprefix="$")
    
    # add the predicted price and trend lines to the chart
    fig.add_trace(go.Scatter(x=forcast_processed.ds, y=forcast_processed.predicted_price, line=dict(color='#B111D6', width=1), name='Predicted Price'))
    fig.add_trace(go.Scatter(x=forcast_processed.ds, y=forcast_processed.trend, line=dict(color='#0074BA', width=1), name='Predicted Trend'))
    fig.add_trace(go.Scatter(x=forcast_processed.ds, y=forcast_processed.upper_band, line=dict(color='#1E82CD', width=2), name='upper_band'))
    fig.add_trace(go.Scatter(x=forcast_processed.ds, y=forcast_processed.lower_band, line=dict(color='#1E82CD', width=2), name='lower_band'))
    return fig



In [69]:
from datetime import date
# get todays date
today = date.today()
# print("Today's date:", today)

# get the date 2 years ago
two_years_ago = today.replace(year=today.year - 2)
# print("Two years ago:", two_years_ago)

def process_chart_pipeline(symbol):
    data = get_data(symbol)
    # get date 2 years ago
    two_years_ago = TODAYS_DATE - timedelta(days=730)
    data = splice_data(data, two_years_ago)
    forcasting_prep = forcasting_preparation(data)
    forecast = forecast_data(forcasting_prep)
    processed_forecast = process_forecasted_data(forecast)

    # visulize the data 
    fig = plotly_visualize_forecast(symbol, data, processed_forecast)
    return fig

In [ ]:
process_chart_pipeline('C:USDCAD')

## new testing section for the notebook
